## Importação e normalização dos dados

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

clientes = pd.read_excel('base.xlsx', header=1, sheet_name='CLIENTES')
faturamento = pd.read_excel('base.xlsx', header=1, sheet_name='FATURAMENTO ')
churn = pd.read_excel('base.xlsx', header=1, sheet_name='CHURN')

clientes.columns = clientes.columns.str.lower()
faturamento.columns = faturamento.columns.str.lower()
churn.columns = churn.columns.str.lower()


## Criar uma Tabela Dinâmica que mostre o faturamento mensal por segmento e por estado.

In [ ]:
df_completo = pd.merge(faturamento, clientes, on='cliente_id', how='left')

df_completo.columns = df_completo.columns.str.lower()

tabela_dinamica = pd.pivot_table(
     df_completo,
     values='r$ receita',
     index=['segmento', 'estado'],
     columns=['mes_ano'],
     aggfunc='sum',
     fill_value=0,
     margins=True,
     margins_name='TOTAL'
)
tabela_dinamica

##  Liste os 10 clientes que tiveram maior faturamento no último mês

In [ ]:
ultimo_mes = faturamento['mes_ano'].max()

faturamento_ultimo_mes = faturamento[faturamento['mes_ano'] == ultimo_mes].copy()

df_ultimo_mes = pd.merge(
    faturamento_ultimo_mes,
    clientes,
    on='cliente_id',
    how='left'
)
df_ultimo_mes.columns = df_ultimo_mes.columns.str.lower()

top10 = df_ultimo_mes.sort_values(by='r$ receita', ascending=False).head(10)

resultado = top10[['mes_ano', 'estado', 'r$ receita', 'cliente_id', 'segmento']]

resultado

## Calcule a margem bruta por cliente no último mês (margem_bruta = (receita - custo) / receita).

In [ ]:
df_margem = pd.merge(
    faturamento,
    clientes,
    on='cliente_id',
    how='left'
)
df_margem.columns = df_margem.columns.str.lower()

df_margem['margem'] = (df_margem['r$ receita'] - df_margem['r$ custo']).round(2)
df_margem['margem_%'] = (df_margem['margem'] / df_margem['r$ receita'] * 100).round(2)
df_margem


## Identifique a taxa de churn por segmento de empresa no último mês.


In [ ]:
ultimo = churn['mes_ano'].max()
meses = sorted(churn['mes_ano'].unique())
anterior = meses[meses.index(ultimo) - 1] if meses.index(ultimo) > 0 else None

if anterior:
    ativos = (churn[(churn['mes_ano'] == anterior) & (churn['status'] == 'Ativo')]
    .merge(clientes, on='cliente_id')
    .groupby('segmento').size())
else:
    ativos = clientes.groupby('segmento').size()

cancelamentos = (churn[(churn['mes_ano'] == ultimo) & (churn['status'] == 'Cancelado')]
    .merge(clientes,on='cliente_id')
    .groupby('segmento').size())

df = pd.DataFrame({
    'segmento' : ativos.index,
    'ativos_inicio' : ativos.values,
    'cancelamentos' : [cancelamentos.get(s, 0) for s in ativos.index]
})

df['taxa_churn_%'] = (df['cancelamentos'] / df['ativos_inicio'] * 100).round(2)

df


## 5. Criar uma fórmula (SE + PROCV ou ÍNDICE/CORRESP) para classificar os clientes conforme a margem bruta em:
 * Alto valor (Margem > 50%)
 * Médio valor (Margem entre 30% e 50%)
 * Baixo valor (Margem < 30%)

In [ ]:
def classificar_margem(df):
    df['categoria'] = pd.cut(
        df['margem_%'],
        bins=[-float('inf'), 30, 50, float('inf')],
        labels=['Baixo valor', 'Médio valor', 'Alto valor'],
    )
    return df

df_margem = classificar_margem(df_margem)
df_margem

## 6. Construir um gráfico de tendência da receita nos últimos 6 meses.

In [ ]:
ultimos_6_meses = faturamento.groupby('mes_ano')['r$ receita'].sum().sort_index().tail(6)

plt.figure(figsize=(12, 6))
plt.plot(ultimos_6_meses.index,
         ultimos_6_meses.values,
         marker='o',
         linewidth=2,
         markersize=8)

plt.title('Tendência da Receita - Últimos 6 Meses',
          fontsize=14,
          fontweight='bold')

plt.xlabel('Mês', fontsize=12)
plt.ylabel('Receita (R$)', fontsize=12)

for i, valor in enumerate(ultimos_6_meses.values):
    plt.text(i, valor, f'R${valor:,.0f}',
             ha='center', va='bottom',
             fontsize=10)

plt.gca().yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'R${x:,.0f}'))
plt.grid(True, alpha=0.3, linestyle='--')
plt.tight_layout()

# Tabela

tabela = pd.DataFrame({
    'Mês': ultimos_6_meses.index,
    'Receita': ultimos_6_meses.values
})

tabela['Receita'] = tabela['Receita'].apply(lambda x: f'R$ {x:,.2f}')
print(tabela.to_string(index=False))
